In [15]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import TextLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser


def save_response_to_file(response, filename='company_summary.txt'):
    with open(filename, 'w') as file:
        file.write(response)
    print(f"Response has been saved to {filename}")

prompt = ChatPromptTemplate.from_template(
"""
Provide me the accurate answers regarding the context.
<context>
{context}
<context>
Question:{input}
"""
)

# Set up the language model and embeddings
model = Ollama(model='llama3.1')
ollama_embeddings = OllamaEmbeddings(
    model='llama3.1',
    model_kwargs={'device': 'gpu'}
)

# Define the function to process documents and extract information
def response_from_doc(loader, input_prompt):
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    final_doc = text_splitter.split_documents(document)

    vectors = FAISS.from_documents(final_doc, ollama_embeddings)
    retriever = vectors.as_retriever()

    document_chain = create_stuff_documents_chain(model, prompt)
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    response = retrieval_chain.invoke({'input': input_prompt})
    return response['answer']

# Define prompts for extracting information


prompt_for_candidate_info = """


    Please provide a detailed summary of the candidate's professional profile from the resume. Focus on the following aspects:

    1. Candidate’s Name: The full name of the candidate.
    2. Candidate’s Designation: The current designation of the candidate that would be clearly mentioned in the resume.
    3. Industry: The industry or field in which the candidate has worked.
    4. Professional Summary: A concise overview of the candidate’s expertise, key skills, and areas of specialization.
    5. Key Skills: A list of the candidate's main technical and soft skills.
    6. Work Experience: Highlights of the candidate's most recent work experience, including notable projects and accomplishments.
    7. Relevant Projects: Information about significant personal or professional projects that demonstrate the candidate's capabilities and achievements.
    8. Achievements: Notable accomplishments or achievements listed in the document.
    9. Career Objectives: The candidate’s professional goals and what they are looking for in their next role.
    10. Linkedin Profile: Linkedin Profile Link that will be like https://www.linkedin.com/candidate-unique-identifier/.


    Provide the extracted information in the following format:

    Candidate’s Name: [Name]
    Industry: [Industry]
    Key Skills: [Skill 1], [Skill 2], [Skill 3]
    Achievements: [Achievement 1], [Achievement 2]
    Current Role: [Role] at [Current Company]
    Relevant Projects: Information about significant personal or professional projects that demonstrate the candidate's capabilities and achievements.
    Linkedin Profile Link: If there is linkedin profile link then return the link else do not return.

    Make sure to carefully review the document and provide accurate and complete details as thee document is divided in two sections.
    one is professional projects and another is personal projects.
    Do not return howm many years of experience a candidate have.
        """


prompt_for_contract_info = """
I have a job description, and I need to extract key information to create a proposal email. Please identify and extract the following details from the job description:

1. **Job Title**: What is the title of the job?
2. **Key Responsibilities**: What are the main responsibilities of the role?
3. **Required Skills**: What specific skills and qualifications are required for this role?
4. **Preferred Qualifications**: Are there any additional preferred qualifications or experiences mentioned?
5. **Application Instructions**: Are there specific instructions for applying or any restrictions on third-party agencies?


"""

prompt_for_email = ChatPromptTemplate.from_messages([
    ('system', '''You are an expert in crafting professional proposals for job applications. 
     Your task is to create a compelling and personalized proposal based on the candidate’s profile and the job description provided. 

1. **Extracted Candidate Information**:
   - Candidate’s Name: [Name]
   - Industry: [Industry]
   - Key Skills: [Skill 1], [Skill 2], [Skill 3]
   - Achievements: [Achievement 1], [Achievement 2]
   - Current Role: [Role] at [Current Company]
   - Relevant Projects: Information about significant personal or professional projects that demonstrate the candidate's capabilities and achievements.
   - Linkedin Profile Link: [Linkedin Profile Link]

2. **Extracted Job Description Information**:
   - Job Title: [Job Title]
   - Key Responsibilities: [List of Key Responsibilities]
   - Required Skills: [List of Required Skills]
   - Preferred Qualifications: [List of Preferred Qualifications, if any]
   - Application Instructions: [Any specific application instructions or restrictions]

Your proposal should include:

1. **Introduction**:
   - Address the proposal to the hiring manager or relevant person.
   - State the purpose of the proposal and briefly mention the candidate's interest in the role.

2. **Alignment with Job Requirements**:
   - Describe how the candidate’s skills, achievements, and relevant projects align with the key responsibilities and required skills for the job.
   - Highlight any specific achievements or projects that are particularly relevant to the role.

3. **Candidate’s Fit**:
   - Explain why the candidate is a strong fit for the position, emphasizing their relevant experience and how it meets the needs outlined in the job description.

4. **Closing Statement**:
   - Express enthusiasm about the opportunity and a willingness to discuss the candidate’s fit for the role in more detail.

5. **Note**:
   - Include a note that this proposal content was generated by an automation script and provide a link to the GitHub repository showcasing this work. The GitHub repo link is: https://github.com/bachan007/cold_email_generator

Here is an example structure for the proposal:

---

**Subject: Proposal for [Job Title] Position**

Dear [Recipient’s Name],

I am excited to submit my proposal for the [Job Title] position. With my background in [Industry] and a proven track record of [mention key achievements or projects], I am confident in my ability to contribute effectively to your team.

Based on the job description, I understand that you are seeking someone with expertise in [mention required skills and responsibilities]. My experience includes [highlight relevant experience or projects], which aligns closely with the needs of the role.

Here is how my background aligns with your requirements:
- **Skills**: [Match candidate's key skills with job requirements]
- **Relevant Projects**: [Detail specific projects or achievements that relate to the job]
- **Achievements**: [Mention notable accomplishments]

I am enthusiastic about the opportunity to bring my skills to [Company/Client] and am eager to discuss how my background fits the needs of your team.

Thank you for considering my proposal. I look forward to discussing this opportunity further.

Best regards,

[Candidate’s Name]  
[Contact Information]  
[LinkedIn Profile or Portfolio URL, if applicable]

---

Please ensure to tailor the proposal based on the specific details extracted from the candidate’s profile and the job description.
'''),
    ('user', 'question:{question}')
])



chain_for_email = prompt_for_email | model | StrOutputParser()


# company_url = 'https://www.krtrimaiq.ai/services/data-engineering/cloud-data-engineering'

# Process resume
portfolio_loader = TextLoader('/home/tranzitamachine/Desktop/dev_env/test/cold_email_generator/company_info/portfolio_doc_hashim.txt')
candidate_info = response_from_doc(portfolio_loader, prompt_for_candidate_info)
print(candidate_info)

# Process company info
jd_loader = TextLoader('/home/tranzitamachine/Desktop/dev_env/test/cold_email_generator/company_info/requirement.txt')
company_info = response_from_doc(jd_loader, prompt_for_contract_info)
print(company_info)

# Generate the cold email
email_prompt = f"""
Please write a professional proposal for a job application based on the following information:
Candidate’s Portfolio, Projects and Skills Highlights:
{candidate_info}
Job description Description:
{company_info}
Email Objective:
- The goal of this email is to introduce the candidate, their skills, their most recent profile overview, and showcase the capability and interest in potential job that match with their recent projects and tech stack at the current company.
Desired Outcome:
- Request a meeting or call to discuss potential opportunities.
Make sure to focus on the candidate's key skills and relevant achievements without mentioning the total years of experience. Align the candidate's expertise with the company's working domain and current trends in the industry.
Additional Note:
- Mention that the content of this email is generated by an automation script, which showcases the candidate's proficiency in leveraging Generative AI (GenAI) technology.. 
- Provide a link to the GitHub repository where the script and other relevant projects are showcased: https://github.com/bachan007/cold_email_generator
"""


response = chain_for_email.invoke({'question': email_prompt})

print(response)

save_response_to_file(response,filename='/home/tranzitamachine/Desktop/dev_env/test/cold_email_generator/company_info/output.txt')

Here are the extracted information:

Candidate’s Name: Mohd Hashim
Industry: Data Engineering
Professional Summary: Experienced and result-oriented Data Engineer with expertise in developing and maintaining complex data pipelines, implementing effective ETL processes, and leveraging Power BI for insightful data visualization.

Key Skills:
- Python, PySpark, Pandas, NumPy
- SQL, Azure Data Factory, Azure Databricks
- Azure Functions, Azure Logic Apps, Power BI

Work Experience: At Tranzita Systems, Mohd Hashim has demonstrated his expertise through significant projects including end-to-end data processing pipelines for major clients such as P&G.

Relevant Projects:
IDAS (Intelligent Decision Automation System): Implemented a robust ETL workflow using Azure Data Factory and developed critical Python logic for scheduling within Azure Function App.
Farm To Shelf, P&G Hyderabad: Spearheaded data pipeline development and interactive dashboard creation in Power BI, leading to improved operati

In [21]:
# cold_emailer for company about us page

import streamlit as st
import tempfile
import os
from io import BytesIO
from langchain_community.llms import Ollama
from langchain_community.document_loaders import TextLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser


def save_response_to_file(response, filename='company_summary.txt'):
    with open(filename, 'w') as file:
        file.write(response)
    print(f"Response has been saved to {filename}")

prompt = ChatPromptTemplate.from_template(
"""
Provide me the accurate answers regarding the context.
<context>
{context}
<context>
Question:{input}
"""
)

# Set up the language model and embeddings
model = Ollama(model='llama3.1')
ollama_embeddings = OllamaEmbeddings(
    model='llama3.1',
    model_kwargs={'device': 'gpu'}
)

# Define the function to process documents and extract information
def response_from_doc(loader, input_prompt):
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    final_doc = text_splitter.split_documents(document)

    vectors = FAISS.from_documents(final_doc, ollama_embeddings)
    retriever = vectors.as_retriever()

    document_chain = create_stuff_documents_chain(model, prompt)
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    response = retrieval_chain.invoke({'input': input_prompt})
    return response['answer']

# Define prompts for extracting information


prompt_for_candidate_info = """


    Please provide a detailed summary of the candidate's professional profile from the resume. Focus on the following aspects:

    1. Candidate’s Name: The full name of the candidate.
    2. Candidate’s Designation: The current designation of the candidate that would be clearly mentioned in the resume.
    3. Industry: The industry or field in which the candidate has worked.
    4. Professional Summary: A concise overview of the candidate’s expertise, key skills, and areas of specialization.
    5. Key Skills: A list of the candidate's main technical and soft skills.
    6. Work Experience: Highlights of the candidate's most recent work experience, including notable projects and accomplishments.
    7. Relevant Projects: Information about significant personal or professional projects that demonstrate the candidate's capabilities and achievements with the used technologies.
    8. Achievements: Notable accomplishments or achievements listed in the document.
    9. Career Objectives: The candidate’s professional goals and what they are looking for in their next role.
    10. Linkedin Profile: Linkedin Profile Link that will be like https://www.linkedin.com/candidate-unique-identifier/.


    Provide the extracted information in the following format:

    Candidate’s Name: [Name]
    Industry: [Industry]
    Key Skills: [Skill 1], [Skill 2], [Skill 3]
    Achievements: [Achievement 1], [Achievement 2]
    Current Role: [Role] at [Current Company]
    Relevant Projects: Information about significant personal or professional projects that demonstrate the candidate's capabilities and achievements.
    Linkedin Profile Link: If there is linkedin profile link then return the link else do not return.

    Make sure to carefully review the document and provide accurate and complete details as thee document is divided in two sections.
    one is professional projects and another is personal projects.
    Do not return howm many years of experience a candidate have.
        """


prompt_for_company_info = """
Please provide detailed information about the company from the website. I am interested in the following aspects:
1. Company Overview: A summary of the company’s history, mission, and vision.
2. Key Areas of Exploration: The primary sectors or industries the company is involved in. Notable projects or focus areas should be mentioned.
3. Products and Services: Details on what products or services the company offers.
4. Company Structure: Any information about the company’s organizational structure, key departments, or teams.
5. Recent Developments: Any recent news or updates related to the company.

Ensure the information is accurate and up-to-date, reflecting the company's current focus and activities.

"""

prompt_for_email = ChatPromptTemplate.from_messages([
    ('system', '''You are an expert in writing professional cold emails for job applications. 
    Your task is to craft a compelling and personalized email based on the provided details. 
    Ensure that the email is professional, concise, and tailored to the company's needs and values. 
    Do not include any mention of the total years of experience of the candidate. 
    Focus on highlighting the candidate's relevant skills and achievements, aligning them with the company's working domain and current trends in the industry. 
    Example 1: If the candidate profile is Azure Data Engineer, and company works in the domain of Data Engineering. Then you need to write a compelling email,
    which showcases the Candidate's capability, skills and achievements in the data engineering domain.
    Example 2: If the candidate profile is Full Stack Developer, and company works in the domain of Web development or full stack development. Then you need to write a compelling email,
    which showcases the Candidate's capability, skills and achievements in the Full Stack domain.
    Example 2: If the candidate profile is Data Engineer, and company works in the domain of Azure Databricks. Then you need to write a compelling email,
    which showcases the Candidate's capability, skills and achievements in the Databricks technology, if the candidate has worked or proven hands on databricks.
    
    So first focus is on candidate's profile, then align with company's service in that area and then retrive the information in about the candidate for the particular domain
    and then generate the content of the email.
    Additionally, include a note about the email content being generated by an automation script and provide a link to the GitHub repository showcasing this work.
    The GitHub repo link is: https://github.com/bachan007/cold_email_generator'''),
    ('user', 'question:{question}')
])



chain_for_email = prompt_for_email | model | StrOutputParser()


company_url = 'https://dtcinfotech.com/platform-solutions/databricks/'

# Process resume
txt_loader = TextLoader('/home/tranzitamachine/Desktop/dev_env/test/cold_email_generator/company_info/portfolio_doc')
candidate_info = response_from_doc(txt_loader, prompt_for_candidate_info)
print(candidate_info)

# Process company info
web_loader = WebBaseLoader(company_url)
company_info = response_from_doc(web_loader, prompt_for_company_info)
print(company_info)

# Generate the cold email
email_prompt = f"""
Please write a cold email for a job application based on the following information:
Candidate’s Resume Highlights:
{candidate_info}
Company Description:
{company_info}
Email Objective:
- The goal of this email is to introduce the candidate, their skills with the projects, their most recent profile overview, and express interest in potential job opportunities that match with their recent projects and tech stack at the current company.
Desired Outcome:
- Request a meeting or call to discuss potential opportunities.
Make sure to focus on the candidate's key skills and relevant achievements without mentioning the total years of experience. Align the candidate's expertise with the company's working domain and current trends in the industry.
Additional Note:
- Mention that the content of this email is generated by an automation script, which showcases the candidate's proficiency in leveraging Generative AI (GenAI) technology.. 
- Do not forget to add linkedin profile.
- Provide a link to the GitHub repository where the script and other relevant projects are showcased: https://github.com/bachan007/cold_email_generator
"""


response = chain_for_email.invoke({'question': email_prompt})

save_response_to_file(response,filename='dtcinfotech.txt')

Here are the extracted information:

**Candidate’s Name:** Bachan Nigam
**Industry:** Data Engineering/Procter & Gamble (Consumer Goods)
**Key Skills:**
	* Programming (Python, R, PySpark)
	* Cloud (Azure Functions, Azure Databricks, Azure Data Factory, Azure Active Directory, ADLS, Azure SQL, Azure Synapse, Logic Apps, Cosmos DB)
	* Soft Skills (Communication, Problem-Solving, Critical Thinking, Leadership, Logical Reasoning)
**Achievements:**
	* Demonstrated expertise in web scraping using requests, scrapy, bs4, pandas, numpy, and pyodbc.
	* Successfully implemented automated solutions for reporting accuracy and optimized inventory management.
	* Achieved 100% record iteration with automated E-commerce order audits using Pandas and KNIME.
**Current Role:** Azure Data Engineer at Tranzita Systems (Lucknow)
**Relevant Projects:**
	1. **BRS (Branch Replenishment System)** - Designed and implemented Databricks pipelines for data integration from ADLS, Azure SQL DB, SharePoint, and DataHu

In [ ]:
prompt_for_candidate_info = """Please extract the following details from the provided resume:

    1. **Candidate’s Name**: The full name of the candidate.
    2. **Candidate’s Designation**: The current designation of the candidate that would be clearly mentioned in the resume.
    3. **Industry**: The industry or field in which the candidate has worked.
    4. **Key Skills**: A list of most important and primary skills mentioned in the resume and align with the latest industry trends in the designation of the candidate.
    5. **Achievements**: Notable accomplishments or achievements listed in the resume.
    6. **Linkedin Profile**: Linkedin Profile Link that will be like https://www.linkedin.com/candidate-unique-identifier/.

    Provide the extracted information in the following format:

    **Candidate’s Name**: [Name]
    **Industry**: [Industry]
    **Key Skills**: [Skill 1], [Skill 2], [Skill 3]
    **Achievements**: [Achievement 1], [Achievement 2]
    **Current Role**: [Role] at [Current Company]
    **Linkedin Profile Link**: If there is linkedin profile link then return the link else do not return.

    Make sure to carefully review the resume and provide accurate and complete details.
    Do not return howm many years of experience a candidate have.
        """

In [ ]:
prompt_for_company_info = """
Please provide detailed information about the company from the website. I am interested in the following aspects:

1. **Company Overview**: A summary of the company’s history, mission, and vision.
2. **Key Areas of Exploration**: The primary sectors or industries the company is involved in. Notable projects or focus areas should be mentioned.
3. **Products and Services**: Details on what products or services the company offers.
4. **Company Structure**: Any information about the company’s organizational structure, key departments, or teams.
5. **Recent Developments**: Any recent news or updates related to the company.

Ensure the information is accurate and up-to-date, reflecting the company's current focus and activities.

"""

In [ ]:
prompt_for_email = ChatPromptTemplate.from_messages([
    ('system', '''You are an expert in writing professional cold emails for job applications. 
    Your task is to craft a compelling and personalized email based on the provided details. 
    Ensure that the email is professional, concise, and tailored to the company's needs and values. Do not include any mention of the total years of experience of the candidate. Focus on highlighting the candidate's relevant skills and achievements, aligning them with the company's working domain and current trends in the industry.'''),
    ('user', 'question:{question}')
])

In [ ]:
chain_for_email = prompt_for_email | model | StrOutputParser()